In [2]:
from pprint import pprint 
from openpyxl.workbook.workbook import Workbook 
import requests 
import json 
from datetime import datetime 

# 헤더값은 각자의 f12의 내용 참고해서 작성해야됨. 아래 코드의 값과 다를 수 있음. 
header = { 
    'accept': '*/*', 
    'accept-encoding': 'gzip, deflate, br', 
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7', 
    'cookie': 'mid=YlkSBQALAAHicDHRHF0ryc2Vklbr; ig_did=1217AE7F-5A0D-4572-A3B2-8667E9E88EF3; ig_nrcb=1; csrftoken=JQg9fvzgRGiIQfkXP1goVs2RuUc1wnlA; ds_user_id=49734310131; sessionid=49734310131%3Ar6OszlzRoUbT05%3A9; shbid="11165\05449734310131\0541681540526:01f74c66e867430b3616d511d732727e338617939647e3cd380ce36b76a1d9a2622dd839"; shbts="1650004526\05449734310131\0541681540526:01f763311a1fe9b02c4b4288bd0095024b37beecb31c5616aa20e6d98ddfc44e594150d2"; rur="EAG\05449734310131\0541681779288:01f7b282118cd865e8cd2d62ac46beeb016d5befa4295347adc5703584d0978e2688889d"', 
    'referer': "https://www.instagram.com/", 
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="100", "Google Chrome";v="100"', 
    'sec-ch-ua-mobile': '?0', 
    'sec-ch-ua-platform':'windows', 
    'sec-fetch-dest': 'empty', 
    'sec-fetch-mode': 'cors', 
    'sec-fetch-site': 'same-origin', 
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36', 
    'x-asbd-id': '198387', 
    'x-ig-app-id': '936619743392459', 
    'x-ig-www-claim': 'hmac.AR2nMlqUjTOj2hZlRuhqHFwlXDF0Qck69oM_gSbxidPKXob7', 
    'x-requested-with': 'XMLHttpRequest' 
    } 


dataList = [] 
URL = 'https://www.instagram.com/explore/tags/제주/?__a=1' 

# 개수확인용도 
pidCount = 0 

while(True): 

    res = requests.get(URL, headers =header) 
    res = res.json() 

    if 'next_page' not in res['data']['recent'].keys() or int(res['data']['recent']['next_page']) == 0: break 
    max_id = res['data']['recent']['next_max_id'] 


    for n in res['data']['recent']['sections']: 
        for m in ((n['layout_content']['medias'])): 
            # m : 인스타 게시글 1개 
            m = m['media'] 
            data = {} 
            data['pagePk'] = m['code'] 
            data['URL'] = 'https://www.instagram.com/p/'+ data['pagePk']+"/" 
            data['user_name'] = (m['user']['username']) 

            # 이미지/동영상 체크 
            keys = m.keys() 
            if 'carousel_media' in keys: 
                data['image'] = (m['carousel_media']) 
            else: 
                data['image'] = [{'image_versions2':{"candidates":[{'url'}]}}] 
                # data['image'] = [{'image_versions2':{"candidates":[{'url':'동영상업로드'}]}}] 
            
            dataList.append(data) 
            pidCount = pidCount+1 
            # pprint (pidCount) 
            comments = m['comments']
            moreComments = m['has_more_comments']
            # print(len(m['comments']))        # 댓글 부분

            # for i in range(1,20):
            #     try:
            #         print(comments[i])
            #     except:
            #         print(0)
            
            # print(keys)
            print("댓글더보기?")
            print(moreComments)

            if (len(comments) != 0):
                print(len(comments))
                print(comments)
            elif (len(comments) ==0):
                print("none")


            # print(m['comments'])        # 댓글 부분
            # print('\n')
            # print(res['data']['description'])

            # pprint (comment) 
            # 개수확인용도 

    URL = 'https://www.instagram.com/explore/tags/'+'태그명'+'/?__a=1&max_id='+max_id 
    
# 엑셀 다운로드 및 이미지 저장 

from openpyxl import Workbook 
wb = Workbook() 
ws = wb.active 


#시트생성 
sheet1 = wb['Sheet'] 
sheet1.title = '수집 데이터' 
sheet1['A1'] = '아이디' 
sheet1['B1'] = 'URL' 
sheet1['C1'] = '이미지' 

sheetIndex = 2 

for data in dataList: 
    sheet1['A'+str(sheetIndex)] = data['user_name'] 
    sheet1['B'+str(sheetIndex)] = data['URL'] 

    # 이미지링크 엑셀에 저장 
    for imgUrl in data['image']: 
        url = imgUrl['image_versions2']['candidates']['url'] 
        # url = imgUrl['image_versions2']['candidates'][0]['url'] 
        sheet1['C'+str(sheetIndex)] = url 

        # 이미지 폴더에 저장 
        if not os.path.exists("./images/"+data['user_name']): 
            os.mkdir("./images/"+data['user_name']) 
            
        if url not in '동영상업로드': 
            r = requests.get(url) 
            file = open("./images/"+data['user_name']+'/'+data['user_name']+str(sheetIndex)+".jpg","wb") 
            file.write(r.content) 
            file.close() 
            
        sheetIndex = sheetIndex+1 

wb.save('./test1.xlsx')


댓글더보기?
False
none
댓글더보기?
False
2
[{'pk': 17938021694049655, 'user_id': 640890840, 'text': '🤍', 'type': 0, 'created_at': 1650270924, 'created_at_utc': 1650270924, 'content_type': 'comment', 'status': 'Active', 'bit_flags': 0, 'did_report_as_spam': False, 'share_enabled': False, 'user': {'pk': 640890840, 'username': 'k_songju', 'full_name': '송주', 'is_private': False, 'profile_pic_url': 'https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-19/277290461_4827882924000741_1371538184619305835_n.jpg?stp=dst-jpg_s150x150&_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=109&_nc_ohc=WJ5JBv-qKEYAX98cacL&tn=QI2I8rJliEO4IJr_&edm=ABZsPhsBAAAA&ccb=7-4&oh=00_AT9bqo1Z5yoNBKX-hI70oJ2E45kDL0HF30aujpd99-jJ6A&oe=62631EE4&_nc_sid=4efc9f', 'profile_pic_id': '2802561692620193722_640890840', 'is_verified': False, 'follow_friction_type': -1, 'growth_friction_info': {'has_active_interventions': False, 'interventions': {}}, 'account_badges': []}, 'is_covered': False, 'media_id': 2818999236952373766, 'private_repl

TypeError: list indices must be integers or slices, not str